In [3]:
import train
import coco_dataset
import transforms as T

main_folder_path = "../"
model_name = "tf_efficientdet_d0"
target_dim = 512
num_classes, train_df, test_df, categories_df = train.process_data_coco(main_folder_path, None)

dataset = coco_dataset.COCODataset(train_df, True, model_name, main_folder_path, target_dim, num_classes,
                                        T.get_transform(train=True), False, False)
# dataset_test = coco_dataset.COCODataset(test_df, False, model_name, main_folder_path, target_dim, num_classes,
#                                              T.get_transform(train=True), False)

loading annotations into memory...
Done (t=11.78s)
creating index...
index created!
loading annotations into memory...
Done (t=0.36s)
creating index...
index created!
Num of classes is [90]
[{'supercategory': 'person', 'id': 1, 'name': 'person'}, {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'}, {'supercategory': 'vehicle', 'id': 3, 'name': 'car'}, {'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'}, {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'}, {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'}, {'supercategory': 'vehicle', 'id': 7, 'name': 'train'}, {'supercategory': 'vehicle', 'id': 8, 'name': 'truck'}, {'supercategory': 'vehicle', 'id': 9, 'name': 'boat'}, {'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'}, {'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'}, {'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'}, {'supercategory': 'outdoor', 'id': 14, 'name': 'parking meter'}, {'supercategory': 'outdoor', 'id': 15, 'name'

In [6]:
import visualize
import torch
import utils
import time

device = 'cuda:0'
vis = visualize.Visualize(main_folder_path, categories_df, target_dim, dest_folder=None)

def show_image_data_ground_truth(dataset, idx, is_colab, figsize=(40, 40)):
    image, target = dataset.__getitem__(idx)
    vis.show_image_data(image, torch.sub(target['labels'], 1), target['masks'], target['boxes'], figsize=figsize)

    
data_loader = torch.utils.data.DataLoader(
            dataset, batch_size=7, shuffle=False, num_workers=2,
            collate_fn=utils.collate_fn)

model = train.get_model_instance_segmentation_efficientnet(model_name, num_classes, target_dim, freeze_batch_norm=True)
model.to(device)
for images, targets in data_loader:
    images_gpu = list(img.to(device) for img in images)
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    model_time = time.time()
    outputs = model(images_gpu, 0.0001)

    # targets_cpu = [{k: v.to(cpu_device) if torch.is_tensor(v) else v for k, v in t.items()} for t in targets]
    outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
    model_time = time.time() - model_time
    print("Memory usage [{}]".format(train.get_memory_usage()))
    if train.get_memory_usage() > 70.0:
        break

# print("Memory usage at iteration [{}] is [{}]".format(0, train.get_memory_usage()))
# for i in range(20):
#     show_image_data_ground_truth(dataset, i, False)
# #     img, tgt = dataset.__getitem__(i)
#     print("Memory usage at iteration [{}] is [{}]".format(i, train.get_memory_usage()))

Using EffDet detection model
Freezing batch normalization weights


NameError: name 'box_threshold' is not defined